In [1]:
# coding:utf-8
import argparse
#===============================================================
import os,sys,glob,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
from scipy.stats import ttest_ind,linregress;
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import matthews_corrcoef
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
from sklearn.metrics import auc, mean_absolute_error, mean_squared_error, precision_recall_curve, \
r2_score,roc_auc_score, accuracy_score, log_loss
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from sklearn.metrics import f1_score,confusion_matrix

In [2]:
h1_list = ['r2_score','pearson_r2','rmse','mse','mae','roc','prc','p-stat','Recall',
'Precision','f1','BA','accuracy', 'TN', 'FP', 'FN','TP','SP','SE','NPV','MCC']

In [3]:
def polyfit(x, y, degree):
	#coeffs = numpy.polyfit(x, y, degree) # Polynomial Coefficients
	correlation = numpy.corrcoef(x, y)[0,1]
	return correlation**2

def calc_class_roc_prc(y_true,y_pred,pos_label=1):
	#print('y_true =', y_true)
	#y_ = [y for y in y_true if y not in [0,1]]
	#print('y_ = ',y_)   ## -9223372036854775808, must be NaN
	x_roc=roc_auc_score(y_true,y_pred)
	if pos_label==0:x_roc = 1 - x_roc		
	x_precision, x_recall, x_thresholds = precision_recall_curve(y_true,y_pred)
	x_prc = auc(x_recall, x_precision)
	return {'roc':x_roc,'prc':x_prc}

def calc_class_other_stats_with_threshold(y_true, y_pred, threshold=0.5):
	ind0=np.where(y_true==0)[0];ind1=np.where(y_true==1)[0]
	y_pred0=y_pred[ind0];y_pred1=y_pred[ind1];
	p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
	#print('p_stat = ', p_0_1)
	y_hard_pred = [1 if p > threshold else 0 for p in y_pred] # binary prediction
	#x_cohen_kappa = cohen_kappa_score(y_true, y_pred, weights='linear')	
	### not understand, can problem, Can't handle mix of binary and continuous
	x_cohen_kappa = cohen_kappa_score(y_true, y_hard_pred, weights='linear')
	## True and false values
	TN, FP, FN, TP = confusion_matrix(y_true, y_hard_pred).ravel()	
	#TP = sum((y_true == 1) & (y_hard_pred == 1))
	#TN = sum((y_true == 0) & (y_hard_pred == 0))
	#FN = sum((y_true == 1) & (y_hard_pred == 0))
	#FP = sum((y_true == 0) & (y_hard_pred == 1))
	# SE (Sensitivity), hit rate, recall, or true positive rate
	x_Recall = TP/(TP+FN)
	SE=x_Recall
	# Precision or positive predictive value
	x_Precision = TP/(TP+FP)
	#f1_score = (2*x_Precision*x_Recall)/ (x_Precision + x_Recall)
	f1 = f1_score(y_true, y_hard_pred)
	# CCR (Correct classification rate), BA (balanced accuracy)
	# Specificity or true negative rate
	SP = TN/(TN+FP) 
	x_BA = (SE + SP)/2
	x_accuracy = accuracy_score(y_true, y_hard_pred)
	# Negative predictive value
	NPV = TN/(TN+FN)
	x_MCC = matthews_corrcoef(y_true, y_hard_pred)
	other_stats = {'p_stat':p_0_1,'Recall':x_Recall,'Precision':x_Precision, 'f1':f1,'BA':x_BA,
	'accuracy':x_accuracy, 'TN':TN, 'FP':FP, 'FN':FN,'TP':TP,'SP':SP,'SE':SE,
	'NPV':NPV,'MCC':x_MCC,'cohen_kappa':x_cohen_kappa}  # 
	return other_stats

In [4]:
def calc_ef_threholds(y_true,y_pred,ef_threholds=[0.01,0.05,0.1]):
	# y_true,y_pred are np.array type
	df = pd.DataFrame()   ## merge  y_true,y_pred as df 's  two cols
	df['y_true'] = y_true;df['y_pred'] = y_pred
	n_actives = len(df[df['y_true']==1].index)
	n_total = len(df.index)
	random_rate = n_actives/ n_total
	df.sort_values(by='y_pred',ascending=False,inplace=True)
	EFs = {}
	for ef_threhold in ef_threholds:
		screen_range = int(np.ceil(n_total * ef_threhold))
		screen_rate = sum(df['y_true'][:screen_range]) / screen_range
		#print('n_actives,n_total = ',n_actives,n_total)
		print('random_rate,screen_range,screen_rate = ', random_rate,screen_range,screen_rate)
		EF = screen_rate / random_rate
		EF_name = 'EF_{}'.format(ef_threhold)
		#print('{} = {}'.format(EF_name,EF))
		EFs.update({EF_name:EF})
	return EFs

In [5]:
def list_stat_for_class(true_file,pred_file,pos_label,ef_threholds):
    t_df = pd.read_csv(true_file)
    p_df = pd.read_csv(pred_file)
    #t_values = t_df[label_col].values
    t_values = t_df.iloc[:,1].values.astype(int)
    if model == 'DMPNN': p_values = p_df.iloc[:,1].values
    else: p_values = p_df.iloc[:,0].values
    ind = np.where((t_values==0) | (t_values==1))[0]	 ### to remove those NaN label
    #print('ind = ',ind)
    y_true = t_values
    [ind]
    y_pred = p_values[ind]
    print('y_true[:10], y_pred[:10] =',y_true[:10], y_pred[:10])
    dic = {}
    dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
    dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
    dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
    w_line= pred_file +',' + ','.join([str(dic[x]) for x in header])+'\n'
    f = open(eva_csv_name, 'a'); f.write(w_line); f.close()
    return 

In [6]:
efs=ef_threholds=[0.01,0.02,0.05]
thre=threshold=0.5
pos_label = 1

# write header for eva_all.csv  

In [7]:
fingerprint_list = ['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP', 
'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP', 'MHFP6', 'MAP4']
descriptor_list = ['Property', 'Constitution', 'Autocorr', 'Fragment', 'Charge', 
'Estate', 'MOE', 'Connectivity', 'Topology', 'Kappa', 'Path', 'Matrix', 'InfoContent']

models = ['xgb','fcnn','log','svm','rf','DMPNN']
num_folds = 5
#true_file = 'hide/rand_check/fold_0/train_full.csv'
b_list = ['train','val','test','Ext']   ## DMPNN use this

true_file_pat = 'rand_check_{feature}/fold_{i}/{b}_full.csv' 
true_file_e = 'Ext.csv'

pred_file_pat = '{model}/p_{b}_{feature}_fold{i}.csv'  # include DMPNN

In [8]:
!rm eva_with_rmSim.csv

In [9]:
eva_csv_name = 'eva_with_rmSim.csv'
dic = {}
y_true,y_pred =np.array([1,0]), np.array([0.9,0.1])
dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
header=list(dic.keys()); #header=sorted(header)
tmp = [x for x in h1_list if x in header] + [x for x in header if x not in h1_list]
header = tmp
print('header=',header)
f = open(eva_csv_name, 'a');f.write('col_names,' + ','.join(header)+'\n');f.close()

random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
header= ['roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE', 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']


/tmp/ipykernel_2560317/448253304.py:19: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


# eva train,val,test,Ext for ['xgb','fcnn','log','svm','rf']

In [10]:
fps= fingerprint_list+descriptor_list
for feature in fps:
    for model in models:
        print(f'processing feature: {feature}; model: {model}')
        for b in b_list: 
            for i in range(num_folds):
                if b=='Ext': true_file = 'Ext.csv'
                else: true_file = f'rand_{feature}/fold_{i}/{b}_full.csv'             
                pred_file = f'{model}/p_{b}_{feature}_fold{i}.csv'  
                # except DMPNN, other model use this
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)  

processing feature: MorganFP; model: xgb
true_file,pred_file =  rand_MorganFP/fold_0/train_full.csv xgb/p_train_MorganFP_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.94875664 0.8423904  0.7637382  0.9992472  0.00115235 0.9898977
 0.52000916 0.9989837  0.99757236 0.99762136]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_MorganFP/fold_1/train_full.csv xgb/p_train_MorganFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.23679107 0.99803156 0.9692255  0.99751866 0.9992662  0.9805941
 0.9915981  0.9918926  0.9991054  0.01784942]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_MorganFP/fold_2/train_full.csv 

true_file,pred_file =  rand_MorganFP/fold_2/train_full.csv fcnn/p_train_MorganFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.15715094 0.99065137 0.99882096 0.9997991  0.99440986 0.9989028
 0.24395576 0.9140967  0.97184324 0.983808  ]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_MorganFP/fold_3/train_full.csv fcnn/p_train_MorganFP_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.17453946 0.96866846 0.906713   0.99631613 0.98000985 0.08561373
 0.9976546  0.98983204 0.9993869  0.5412671 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_MorganFP/fold_4/train_full.csv fcnn/p_train_MorganFP_fold4.csv
y_true

true_file,pred_file =  rand_MorganFP/fold_4/train_full.csv log/p_train_MorganFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.98116181 0.95632057 0.95217659 0.89718207 0.00354472 0.98295836
 0.97952456 0.04323122 0.14247735 0.00716142]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_MorganFP/fold_0/val_full.csv log/p_val_MorganFP_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.96661842 0.59505242 0.70742189 0.71235552 0.9996798  0.97308897
 0.93046825 0.99925021 0.97067238 0.97323915]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
true_file,pred_file =  rand_MorganFP/fold_1/val_full.csv log/p_val_MorganFP_fold1.csv
y_true[:10], y_pred

random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_MorganFP/fold_2/val_full.csv svm/p_val_MorganFP_fold2.csv
y_true[:10], y_pred[:10] = [1 1 0 0 1 1 1 1 0 1] [0.26844988 0.87000193 0.0694487  0.26607491 0.96977681 0.99293103
 0.85733542 0.97153828 0.29499443 0.99138017]
random_rate,screen_range,screen_rate =  0.7861271676300579 4 1.0
random_rate,screen_range,screen_rate =  0.7861271676300579 7 1.0
random_rate,screen_range,screen_rate =  0.7861271676300579 18 1.0
true_file,pred_file =  rand_MorganFP/fold_3/val_full.csv svm/p_val_MorganFP_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.85879594 0.85948813 0.96518726 0.92880189 0.67175957 0.90777361
 0.99999718 0.99573062 0.97264748 0.73453424]
random_rate,screen_range,screen_rate =  0.7803468208092486 4 1.0
random_rate,screen_range,screen_rate =  0.78034

true_file,pred_file =  rand_MorganFP/fold_1/test_full.csv rf/p_test_MorganFP_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 1 0 0] [0.95733814 0.966255   0.656      0.95931916 0.24662011 0.05
 0.06       0.82873302 0.27462011 0.754     ]
random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
true_file,pred_file =  rand_MorganFP/fold_2/test_full.csv rf/p_test_MorganFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 0 1 1 1 1 1] [0.408      0.994      0.8728783  0.30115931 0.028      0.886
 1.         0.96057058 0.926      0.91981318]
random_rate,screen_range,screen_rate =  0.7780979827089337 4 1.0
random_rate,screen_range,screen_rate =  0.7780979827089337 7 1.0
random_rate,screen_range,screen_rate =  0.7780979827089337 18 1.0
true_file,pred_file =  rand_MorganFP/fold_3/test_full.csv rf/p_test_MorganFP_fold3.csv
y_true[:10], y_pred[:10] = [1 0 1 

...

random_rate,screen_range,screen_rate =  0.8022727272727272 5 0.8
random_rate,screen_range,screen_rate =  0.8022727272727272 9 0.8888888888888888
random_rate,screen_range,screen_rate =  0.8022727272727272 22 0.9545454545454546
true_file,pred_file =  Ext.csv xgb/p_Ext_InfoContent_fold2.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9855769  0.99992764 0.6642145  0.9938135  0.9958022  0.99911946
 0.25790107 0.961502   0.9986915  0.9952472 ]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
true_file,pred_file =  Ext.csv xgb/p_Ext_InfoContent_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9804285  0.9998388  0.9406495  0.9981274  0.99126816 0.98950046
 0.40423256 0.9950706  0.9963231  0.9990392 ]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 0.

true_file,pred_file =  rand_InfoContent/fold_1/train_full.csv log/p_train_InfoContent_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.95307973 0.81414238 0.60482563 0.84884281 0.78070003 0.52609968
 0.38914312 0.82361696 0.70244742 0.21182561]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 0.9642857142857143
random_rate,screen_range,screen_rate =  0.7771758757674251 139 0.9784172661870504
true_file,pred_file =  rand_InfoContent/fold_2/train_full.csv log/p_train_InfoContent_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.29039184 0.78767498 0.72976122 0.89202001 0.79357448 0.86477113
 0.12460007 0.79607539 0.89374003 0.35249337]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 0.9642857142857143
random_rate,screen_range,screen_rate =  0.7804261466233298 56 0.9464285714285714
random_rate,screen_range,screen_rate =  0.7804261466233298 139 0.9640287769784173
true_file,p

true_file,pred_file =  rand_InfoContent/fold_3/train_full.csv svm/p_train_InfoContent_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.43507083 0.71768465 0.4695487  0.91841129 0.91834768 0.54654634
 0.88542965 0.80082715 0.68729109 0.5758002 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 0.8571428571428571
random_rate,screen_range,screen_rate =  0.7822318526543879 56 0.9107142857142857
random_rate,screen_range,screen_rate =  0.7822318526543879 139 0.9496402877697842
true_file,pred_file =  rand_InfoContent/fold_4/train_full.csv svm/p_train_InfoContent_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.85631281 0.8591489  0.93065584 0.81942067 0.6420152  0.62156972
 0.96816563 0.34600146 0.81608368 0.89970973]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 0.8214285714285714
random_rate,screen_range,screen_rate =  0.7833152762730228 56 0.9107142857142857
random_rate,screen_range,screen_rate =  0.7833152762730228 139 0.9568345323741

random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
true_file,pred_file =  rand_InfoContent/fold_1/val_full.csv rf/p_val_InfoContent_fold1.csv
y_true[:10], y_pred[:10] = [1 0 0 1 1 0 0 1 1 0] [0.96444565 0.56381805 0.67733694 0.72382763 0.988      0.326
 0.13272888 0.994      0.99213377 0.512     ]
random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_InfoContent/fold_2/val_full.csv rf/p_val_InfoContent_fold2.csv
y_true[:10], y_pred[:10] = [1 1 0 0 1 1 1 1 0 1] [0.71624665 0.982      0.45613382 0.05872017 0.942      0.98891543
 0.98       0.994      0.424      0.972     ]
random_rate,screen_range,screen_rate =  0.7861271676300579 4 1.0
random_rate,screen_range,screen_rate =  0.

random_rate,screen_range,screen_rate =  0.7803468208092486 4 1.0
random_rate,screen_range,screen_rate =  0.7803468208092486 7 1.0
random_rate,screen_range,screen_rate =  0.7803468208092486 18 1.0
true_file,pred_file =  rand_InfoContent/fold_4/val_full.csv DMPNN/p_val_InfoContent_fold4.csv
y_true[:10], y_pred[:10] = [1 0 1 0 1 1 0 1 0 1] [0.99990249 0.00935879 0.6341626  0.05436706 0.99999154 0.97364354
 0.26240405 0.99790585 0.93361247 0.99077868]
random_rate,screen_range,screen_rate =  0.7832369942196532 4 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 7 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 18 1.0
true_file,pred_file =  rand_InfoContent/fold_0/test_full.csv DMPNN/p_test_InfoContent_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 1 0 1 1 1 1 1] [4.36727831e-04 5.55680320e-03 9.58576441e-01 6.31595254e-02
 9.08967631e-04 9.90095854e-01 9.98264134e-01 9.82197940e-01
 4.69158888e-01 8.92422080e-01]
random_rate,screen_range,screen_rate =  0.78386167146

# eva Ext_rm_sim for ['xgb','fcnn','log','svm','rf'], 

In [11]:
fps= fingerprint_list+descriptor_list
ext_files=['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for feature in fps:
    for model in models:
        for ext_file in ext_files: 
            for i in range(num_folds):
                ext_base = ext_file.replace('.csv','')
                true_file = ext_file      
                pred_file = f'{model}/p_{ext_base}_{feature}_fold{i}.csv'  
                # except DMPNN, other model use this
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)            

true_file,pred_file =  Ext_rm_sim_0_6.csv xgb/p_Ext_rm_sim_0_6_MorganFP_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.99953854 0.9391022  0.8231854  0.9661447  0.92099696 0.9315784
 0.5371261  0.0142152  0.9315784  0.6862498 ]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv xgb/p_Ext_rm_sim_0_6_MorganFP_fold1.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.9997892  0.914503   0.8655053  0.9094524  0.74310744 0.8544302
 0.70059687 0.01597746 0.8544302  0.4369209 ]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv xgb/p_Ext_rm_sim_0_6_MorganFP_fold2.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.9996057  0.8264492  0.7669405  0.94527686 0.7620123  0.92017835
 0.7051104

true_file,pred_file =  Ext_rm_sim_0_8.csv fcnn/p_Ext_rm_sim_0_8_MorganFP_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.9692007  0.9936719  0.99557483 0.43442044 0.99559844 0.9954672
 0.9110884  0.99897754 0.97797215 0.9977106 ]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.9166666666666666
true_file,pred_file =  Ext_rm_sim_0_8.csv fcnn/p_Ext_rm_sim_0_8_MorganFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.90949744 0.98232585 0.9977703  0.37162903 0.9905262  0.98772395
 0.86984175 0.99726295 0.96298283 0.9975172 ]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.9166666666666666
true_file,pred_file =  Ext_rm_sim_0_9.csv fcnn/p_Ext_rm_sim_0_9_MorganFP_fold0.csv
y_true

true_file,pred_file =  Ext_rm_sim_0_7.csv svm/p_Ext_rm_sim_0_7_MorganFP_fold1.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.97196884 0.9148885  0.9831353  0.93251232 0.92236156 0.05103417
 0.99058635 0.7980058  0.9310086  0.84962682]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 1.0
true_file,pred_file =  Ext_rm_sim_0_7.csv svm/p_Ext_rm_sim_0_7_MorganFP_fold2.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.85247218 0.76041792 0.90384824 0.67721644 0.75390562 0.17142742
 0.9742125  0.6445733  0.93507089 0.88105204]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 1.0
true_file,pred_file =  Ext_rm_sim_0_7.csv svm/p_Ext_rm_sim_0_7_MorganFP_fold3.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 

random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0
true_file,pred_file =  Ext_rm_sim_0_9.csv rf/p_Ext_rm_sim_0_9_MorganFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.95643337 0.84056274 0.77688123 0.936      0.988      0.992
 0.93176294 0.88039047 0.62071238 0.89086674]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0
true_file,pred_file =  Ext_rm_sim_0_6.csv DMPNN/p_Ext_rm_sim_0_6_MorganFP_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.62178212 0.98034155 0.99318898 0.8837027  0.92303431 0.91214561
 0.16023557 0.00130092 0.91279978 0.16061968]
random_rate,screen_range,screen_rate =  0.25 1 1.0
random_rate,screen_range,screen_rate =  0.25 1 1.0
random_rate,screen_ran

random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.9166666666666666
true_file,pred_file =  Ext_rm_sim_0_8.csv xgb/p_Ext_rm_sim_0_8_RDkitFP_fold2.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.9894276  0.9307655  0.9933767  0.28591552 0.9964677  0.9972632
 0.9930789  0.9970503  0.99920183 0.99981004]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv xgb/p_Ext_rm_sim_0_8_RDkitFP_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.9715417  0.63100517 0.9932289  0.25284553 0.99697375 0.99789536
 0.9990435  0.9984964  0.99811435 0.99869823]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6

...

random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 0.9473684210526315
true_file,pred_file =  Ext_rm_sim_0_9.csv rf/p_Ext_rm_sim_0_9_Matrix_fold2.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98       0.958      0.90708692 0.999059   0.926      0.97715931
 0.84024665 0.64       0.97       0.99312495]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0
true_file,pred_file =  Ext_rm_sim_0_9.csv rf/p_Ext_rm_sim_0_9_Matrix_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98758638 0.948      0.91224438 0.996      0.96091019 0.97328098
 0.764      0.708      0.96       0.97651848]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727

random_rate,screen_range,screen_rate =  0.3968253968253968 1 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.0
true_file,pred_file =  Ext_rm_sim_0_8.csv xgb/p_Ext_rm_sim_0_8_InfoContent_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.48556876 0.421066   0.9968765  0.890108   0.82066727 0.9644658
 0.91108525 0.96131974 0.8611215  0.9006551 ]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.9166666666666666
true_file,pred_file =  Ext_rm_sim_0_8.csv xgb/p_Ext_rm_sim_0_8_InfoContent_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.58281034 0.97131145 0.96261483 0.88762414 0.9675854  0.99102426
 0.7977915  0.84425247 0.95261246 0.7736707 ]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate

random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv log/p_Ext_rm_sim_0_6_InfoContent_fold3.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.85099083 0.74799042 0.11624811 0.66192282 0.56734228 0.55581659
 0.00416767 0.05790356 0.55581659 0.79073765]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv log/p_Ext_rm_sim_0_6_InfoContent_fold4.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.79160001 0.73981396 0.1368402  0.62853313 0.53847845 0.55537566
 0.01216713 0.03582828 0.55537566 0.72713818]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_7.csv log/p

true_file,pred_file =  Ext_rm_sim_0_9.csv svm/p_Ext_rm_sim_0_9_InfoContent_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.89722522 0.87433854 0.95614832 0.88818113 0.90059095 0.91521565
 0.76981533 0.76272519 0.51005199 0.85529258]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 0.5
random_rate,screen_range,screen_rate =  0.7727272727272727 8 0.75
random_rate,screen_range,screen_rate =  0.7727272727272727 19 0.8947368421052632
true_file,pred_file =  Ext_rm_sim_0_9.csv svm/p_Ext_rm_sim_0_9_InfoContent_fold2.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.88379738 0.86269108 0.93602022 0.85545283 0.87346889 0.90726497
 0.77272997 0.77295164 0.57013185 0.82122027]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 0.5
random_rate,screen_range,screen_rate =  0.7727272727272727 8 0.75
random_rate,screen_range,screen_rate =  0.7727272727272727 19 0.8947368421052632
true_file,pred_file =  Ext_rm_sim_0_9.csv svm/p_Ext_rm_sim_0_9_InfoContent_fold3.c

y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.1014464  0.91971678 0.95208007 0.23623402 0.89611638 0.12359165
 0.97116494 0.68456787 0.9039036  0.94328934]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.75
true_file,pred_file =  Ext_rm_sim_0_8.csv DMPNN/p_Ext_rm_sim_0_8_InfoContent_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.99663615 0.62983239 0.99359035 0.5788604  0.71225142 0.96948045
 0.61104387 0.34611037 0.81548733 0.98511291]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv DMPNN/p_Ext_rm_sim_0_8_InfoContent_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.99463874 0.98211068 0.9993974  0.86088145 0.95507139 0.979

# eva train,val,test,Ext for DMPNN

In [12]:
fps= fingerprint_list+descriptor_list + ['']
for feature in fps:
    model = 'DMPNN'
    for b in b_list:  # ['train','val','test','Ext']
        for i in range(num_folds):
            if b=='Ext': true_file = 'Ext.csv'
            else: true_file = f'rand_{feature}/fold_{i}/{b}_full.csv' 
            pred_file = f'{model}/p_{b}_{feature}_fold{i}.csv'  # except DMPNN, other model use this
            list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)    

y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.9997943  0.97900438 0.98868632 0.99544191 0.00193149 0.99810934
 0.33687958 0.99817204 0.99924719 0.9988097 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.12928048 0.99868935 0.99909616 0.97853172 0.98720479 0.9986589
 0.99619764 0.99733633 0.99797827 0.00172554]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.01534253 0.99657416 0.99995506 0.99994624 0.99875081 0.99923038
 0.50368971 0.92730534 0.99163169 0.99759227]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.780426146

random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.07148336 0.99940431 0.99452895 0.99975747 0.99996841 0.0938829
 0.99998879 0.99979788 0.99830139 0.39058587]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [9.93338346e-01 9.98553216e-01 9.99931574e-01 9.99987006e-01
 1.64805795e-04 9.71993387e-01 9.99886990e-01 1.25177810e-03
 1.85897853e-02 3.64645850e-04]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
y_true[:10], y_pred[:10] = [1 0 1 1 1 1

y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.99909115 0.17823629 0.94899619 0.99579453 0.999982   0.98986268
 0.99970418 1.         0.03198354 0.99999678]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
y_true[:10], y_pred[:10] = [1 0 0 1 1 0 0 1 1 0] [9.99199450e-01 5.79357743e-02 9.84585106e-01 9.99999642e-01
 9.99997973e-01 6.57666096e-05 1.11023633e-06 9.99160171e-01
 9.99967933e-01 2.77409684e-02]
random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
y_true[:10], y_pred[:10] = [1 1 0 0 1 1 1 1 0 1] [9.99557674e-01 9.99031663e-01 2.57629938e-02 7.63486023e-05
 9.98414874e-01 9.99904037e-01 9.99028802e-01 9.94207144e-01
 3.15695226e-01 9.99999881e-01]
random_rate,screen_range,screen_rate =  

y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99938011 0.73141354 0.99706167 0.95540535 0.99649292 0.87524891
 0.99999559 0.99745899 0.96968096 0.81201077]
random_rate,screen_range,screen_rate =  0.7803468208092486 4 1.0
random_rate,screen_range,screen_rate =  0.7803468208092486 7 1.0
random_rate,screen_range,screen_rate =  0.7803468208092486 18 1.0
y_true[:10], y_pred[:10] = [1 0 1 0 1 1 0 1 0 1] [9.99960303e-01 1.77589955e-03 9.34765279e-01 1.59719199e-01
 9.99947309e-01 9.99455512e-01 2.40925016e-04 9.99525428e-01
 9.67398763e-01 9.97118473e-01]
random_rate,screen_range,screen_rate =  0.7832369942196532 4 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 7 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 18 1.0
y_true[:10], y_pred[:10] = [0 0 1 1 0 1 1 1 1 1] [1.21040277e-04 3.82151850e-03 9.53102648e-01 6.77616596e-02
 8.08964658e-04 9.99241233e-01 9.99873161e-01 9.97522652e-01
 5.30735850e-01 9.96434689e-01]
random_rate,screen_range,screen_rate =  

random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 1 0 0] [0.98892945 0.99442673 0.27492806 0.98032904 0.00246477 0.05654246
 0.16889521 0.97385567 0.00155244 0.93730575]
random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 0 1 1 1 1 1] [0.50901711 0.99665689 0.76207268 0.82981551 0.19848886 0.93766445
 0.95177329 0.97933048 0.81446427 0.74895161]
random_rate,screen_range,screen_rate =  0.7780979827089337 4 1.0
random_rate,screen_range,screen_rate =  0.7780979827089337 7 1.0
random_rate,screen_range,screen_rate =  0.7780979827089337 18 1.0
y_true[:10], y_pred[:10] = [1 0 1 1 0 1 1 1 0 1] [0.09702949 0.99183214 0.5107252  0.996

...

random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 0 1 1 1 1 1] [0.06625918 0.98260951 0.94513607 0.52861869 0.49559104 0.93103546
 0.85982639 0.99609238 0.8723768  0.92091477]
random_rate,screen_range,screen_rate =  0.7780979827089337 4 1.0
random_rate,screen_range,screen_rate =  0.7780979827089337 7 1.0
random_rate,screen_range,screen_rate =  0.7780979827089337 18 1.0
y_true[:10], y_pred[:10] = [1 0 1 1 0 1 1 1 0 1] [0.27117679 0.93503118 0.98883855 0.87013537 0.25695163 0.9925971
 0.97473347 0.99927646 0.01561082 0.8837744 ]
random_rate,screen_range,screen_rate =  0.7694524495677233 4 1.0
random_rate,screen_range,screen_rate =  0.7694524495677233 7 1.0
random_rate,screen_range,screen_rate =  0.7694524495677233 18 1.0
y_true[:10], y_pred[:10] = [1 1 0 1 1 1 1 0 1 1] [9.99381304e-01 9.93282676e-01 2.7600044

y_true[:10], y_pred[:10] = [1 1 0 1 1 1 1 0 1 1] [0.98944086 0.97249156 0.22529249 0.976408   0.00346123 0.94216084
 0.9759078  0.48937175 0.95859331 0.98500615]
random_rate,screen_range,screen_rate =  0.7579250720461095 4 1.0
random_rate,screen_range,screen_rate =  0.7579250720461095 7 1.0
random_rate,screen_range,screen_rate =  0.7579250720461095 18 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.96901101 0.99996257 0.99933881 0.99999845 0.99999714 0.99998116
 0.99879766 0.99693346 0.99987364 0.99437189]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.97849494 0.99993169 0.99928075 0.99999356 0.99999344 0.99998331
 0.99814785 0.98880786 0.99970442 0.99444193]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.802272727272727

random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98316598 0.99866199 0.98973161 0.99980992 0.99981254 0.99885559
 0.98644257 0.96924442 0.99481392 0.98136085]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98839903 0.99995601 0.98610395 0.99999523 0.99999189 0.99995553
 0.99825603 0.98641193 0.9992398  0.99907589]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98467064 0.99951839 0.98541784 0.999

y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99144191 0.99942756 0.97713089 0.99983263 0.99981755 0.99966824
 0.98796892 0.98270518 0.9958753  0.99686265]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.97490203 0.96558559 0.7743383  0.99976176 0.00677925 0.99321377
 0.84262246 0.96187854 0.99999952 0.98940253]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.38575491 0.99610376 0.9964301  0.95708972 0.98489815 0.99844068
 0.97511691 0.9931879  0.99761707 0.17679474]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.77717587576

y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.17978837 0.82813054 0.99361378 0.97727388 0.99908626 0.99772733
 0.33060524 0.97330928 0.9367708  0.93434602]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.30346245 0.99063665 0.75683278 0.99467683 0.89677364 0.13834749
 0.99515015 0.97423345 0.99680185 0.58206338]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.99822909 0.88424617 0.98356414 0.96322203 0.23644614 0.96266246
 0.99899822 0.00455293 0.03616285 0.01000574]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.78331527

In [13]:
fps= fingerprint_list+descriptor_list + ['']
ext_files=['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
model = 'DMPNN'
for feature in fps:
    for ext_file in ext_files: #for b in b_list:  # ['train','val','test','Ext']
        print(f'processing feature: {feature}; model: {model}')
        ext_base = ext_file.replace('.csv','')
        for i in range(num_folds):
            true_file = ext_file
            pred_file = f'{model}/p_{ext_base}_{feature}_fold{i}.csv'  # except DMPNN, other model use this
            list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)    

processing feature: MorganFP; model: DMPNN
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.62178212 0.98034155 0.99318898 0.8837027  0.92303431 0.91214561
 0.16023557 0.00130092 0.91279978 0.16061968]
random_rate,screen_range,screen_rate =  0.25 1 1.0
random_rate,screen_range,screen_rate =  0.25 1 1.0
random_rate,screen_range,screen_rate =  0.25 1 1.0
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.99606067 0.98580593 0.98834103 0.90218538 0.93753046 0.95205843
 0.84851849 0.00385378 0.95279443 0.42231905]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.90053403 0.99665451 0.96779042 0.87492555 0.65979725 0.93652636
 0.00294155 0.04266119 0.93674868 0.25662082]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
y_true[:10],

y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.99666119 0.78863627 0.99897623 0.7703653  0.99992871 0.99999952
 0.99999797 0.99999702 0.99998915 0.99988961]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 0.3333333333333333
random_rate,screen_range,screen_rate =  0.6787330316742082 5 0.4
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.5
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.94610012 0.58649892 0.98438847 0.37490302 0.99780744 0.99999547
 0.99993134 0.99986315 0.99985194 0.99239129]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 0.3333333333333333
random_rate,screen_range,screen_rate =  0.6787330316742082 5 0.4
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.5
processing feature: RDkitFP; model: DMPNN
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99994183 0.99940228 0.96481645 0.99866045 0.99894148 0.99996161
 0.99413115 0.96946299 0.55665588 0.99974042]
random_rate,screen_range,screen_rate =  0.77272727

random_rate,screen_range,screen_rate =  0.3968253968253968 1 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.25
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.99959093 0.99999988 0.99999583 0.99999988 0.99999666 0.98913389
 0.99876738 0.01525146 0.99763644 0.60217142]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.0
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.95662594 0.98197156 0.9607513  0.9900074  0.97343397 0.53034079
 0.934228   0.75492227 0.92155778 0.85654759]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.0
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.99826837 0.99991357 0.99926108 0.99998

y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99956757 0.99937868 0.29744011 0.99998331 0.99996078 0.99981624
 0.97035372 0.94651473 0.99930155 0.94082433]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 0.875
random_rate,screen_range,screen_rate =  0.7727272727272727 19 0.7368421052631579
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99681145 0.99410737 0.03330401 0.99963236 0.99977976 0.99643779
 0.95079815 0.75681317 0.58347011 0.94998825]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 0.25
random_rate,screen_range,screen_rate =  0.7727272727272727 8 0.25
random_rate,screen_range,screen_rate =  0.7727272727272727 19 0.15789473684210525
processing feature: EstateFP; model: DMPNN
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.99703515 0.99810481 0.99808228 0.99321657 0.87859893 0.99100542
 0.24073559 0.22887257 0.98253006 0.86223984]
random_rate,screen_range,screen_rate =  0.25

random_rate,screen_range,screen_rate =  0.6787330316742082 3 0.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 0.4
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.5833333333333334
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.99701583 0.89403123 0.99293995 0.09232347 0.99969006 0.9998405
 0.99994326 0.99996316 0.99960977 0.99220979]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 0.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 0.4
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.5
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.93277037 0.93041986 0.80643135 0.02544419 0.99364436 0.99652326
 0.99880314 0.99862421 0.99362177 0.94636643]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 0.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 0.2
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.25
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.998528   0.85700655 0

...

random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.99814785 0.91280156 0.99928075 0.51196021 0.98402607 0.98426509
 0.95494235 0.98760891 0.97456306 0.99804258]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.98681951 0.79630482 0.99811578 0.4582046  0.97243685 0.92222679
 0.93799949 0.99338609 0.92019159 0.99667299]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 0.9166666666666666
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.97667605 0.8997978  0

random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [9.68093634e-01 5.45587540e-01 3.30623016e-02 4.02592152e-01
 1.80321276e-01 2.46246800e-01 1.58239796e-04 8.37149918e-01
 1.42039090e-01 1.82257712e-01]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.98010319 0.50012934 0.09188943 0.48413247 0.2049533  0.11139608
 0.00178376 0.98875684 0.11391538 0.20062512]
random_rate,screen_range,screen_rate =  0.25 1 1.0
random_rate,screen_range,screen_rate =  0.25 1 1.0
random_rate,screen_range,screen_rate =  0.25 1 1.0
processing feature: Kappa; model: DMPNN
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.34560803 0.99275005 0.98937422 0.94545394 0.98098409 0.09039707
 0.99354869 0.940941

y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99887782 0.98142356 0.99019593 0.99943048 0.99910372 0.99995875
 0.9930647  0.99716431 0.75604498 0.75500661]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99291128 0.97708184 0.99101722 0.99797207 0.99842924 0.99994648
 0.98471892 0.99412918 0.97847134 0.80791676]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9962554  0.94922048 0.99786866 0.99859184 0.99836165 0.99996114
 0.99543411 0.99354583 0.98336512 0.96423787]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.772727272727272

random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 0.5
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.75
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.94705319 0.9932394  0.9801175  0.9974016  0.93968284 0.40356356
 0.99460793 0.86797196 0.98242158 0.92293829]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 0.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 0.5
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.5
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.1014464  0.91971678 0.95208007 0.23623402 0.89611638 0.12359165
 0.97116494 0.68456787 0.9039036  0.94328934]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.75
processing feature: InfoContent; model: DMPNN
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0

random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0


In [14]:
print('eva_csv_name = ', eva_csv_name)
df = pd.read_csv(eva_csv_name)

eva_csv_name =  eva_with_rmSim.csv


In [15]:
col_names = df['col_names'].values
m_list = [];data_list = [];feat_list = [];fold_list = []
for col in col_names:
    m,tmp = col.split('/')
    tmp = tmp.replace('.csv','').replace('p_Ext_rm_sim_0_','p_ExtRmSim0')
    _,dt,ft,fold = tmp.split('_')
    m_list.append(m);data_list.append(dt);feat_list.append(ft);fold_list.append(fold)
df['model'] = m_list
df['data'] = data_list
df['feat'] = feat_list
df['fold'] = fold_list

In [17]:
df.head(5)

,col_names,roc,prc,Recall,Precision,f1,BA,accuracy,TN,FP,...,MCC,EF_0.01,EF_0.02,EF_0.05,p_stat,cohen_kappa,model,data,feat,fold
0,xgb/p_train_MorganFP_fold0.csv,0.996238,0.998955,0.992151,0.977707,0.984876,0.955446,0.976165,554,49,...,0.929228,1.278393,1.278393,1.278393,0.0,0.928672,xgb,train,MorganFP,fold0
1,xgb/p_train_MorganFP_fold1.csv,0.996262,0.998937,0.989777,0.978860,0.984288,0.957611,0.975442,571,46,...,0.928397,1.286710,1.286710,1.286710,0.0,0.928098,xgb,train,MorganFP,fold1
2,xgb/p_train_MorganFP_fold2.csv,0.996212,0.998933,0.993059,0.976787,0.984855,0.954589,0.976165,557,51,...,0.929640,1.281351,1.281351,1.281351,0.0,0.928941,xgb,train,MorganFP,fold2
3,xgb/p_train_MorganFP_fold3.csv,0.996273,0.998963,0.991690,0.977697,0.984644,0.955215,0.975804,554,49,...,0.928156,1.278393,1.278393,1.278393,0.0,0.927636,xgb,train,MorganFP,fold3
4,xgb/p_train_MorganFP_fold4.csv,0.996174,0.998950,0.991240,0.978607,0.984883,0.956453,0.976165,553,47,...,0.929007,1.276625,1.276625,1.276625,0.0,0.928580,xgb,train,MorganFP,fold4


In [18]:
df.tail(5)

,col_names,roc,prc,Recall,Precision,f1,BA,accuracy,TN,FP,...,MCC,EF_0.01,EF_0.02,EF_0.05,p_stat,cohen_kappa,model,data,feat,fold
7035,DMPNN/p_Ext_rm_sim_0_9__fold0.csv,0.877142,0.951462,0.993080,0.831884,0.905363,0.655363,0.839572,27,58,...,0.486887,1.294118,1.294118,1.294118,1.513044e-43,0.404869,DMPNN,ExtRmSim09,,fold0
7036,DMPNN/p_Ext_rm_sim_0_9__fold1.csv,0.901425,0.964824,0.993080,0.827089,0.902516,0.643599,0.834225,25,60,...,0.465042,1.294118,1.294118,1.294118,1.234330e-34,0.378304,DMPNN,ExtRmSim09,,fold1
7037,DMPNN/p_Ext_rm_sim_0_9__fold2.csv,0.865988,0.950980,0.989619,0.810198,0.890966,0.600692,0.812834,18,67,...,0.366595,1.294118,1.294118,1.294118,1.012476e-35,0.274270,DMPNN,ExtRmSim09,,fold2
7038,DMPNN/p_Ext_rm_sim_0_9__fold3.csv,0.842805,0.941455,0.972318,0.828909,0.894904,0.644983,0.823529,27,58,...,0.417225,1.294118,1.294118,1.294118,1.209108e-31,0.365939,DMPNN,ExtRmSim09,,fold3
7039,DMPNN/p_Ext_rm_sim_0_9__fold4.csv,0.867535,0.950479,0.986159,0.811966,0.890625,0.604844,0.812834,19,66,...,0.365776,1.294118,1.294118,1.294118,1.716272e-27,0.282386,DMPNN,ExtRmSim09,,fold4


In [19]:
test_df=df[df['data']=='test']
group_test = test_df.groupby(['model','feat']).mean()['roc']
group_test_std = test_df.groupby(['model','feat']).std()['roc']
ext_df=df[df['data']=='Ext']
group_ext = ext_df.groupby(['model','feat']).mean()['roc']
#group_ext.to_csv('group_model_feat_Ext_roc.csv')
group_ext_std = ext_df.groupby(['model','feat']).std()['roc']

In [21]:
train_df=df[df['data']=='train']
group_train = train_df.groupby(['model','feat']).mean()['roc']
group_train_std = train_df.groupby(['model','feat']).std()['roc']

val_df=df[df['data']=='train']
group_val = val_df.groupby(['model','feat']).mean()['roc']
group_val_std = val_df.groupby(['model','feat']).std()['roc']

In [24]:
dfl = [pd.DataFrame() for i in [6,7,8,9]] 
groupl = [group_val for i in [6,7,8,9]] 
group_stdl = [group_val_std for i in [6,7,8,9]] 
for i,sim in enumerate([6,7,8,9]):
    #dfl[0]=df[df['data']=='train']
    dfl[i]=df[df['data']==f'ExtRmSim0{sim}']
    groupl[i] = dfl[i].groupby(['model','feat']).mean()['roc']
    group_stdl[i] = dfl[i].groupby(['model','feat']).std()['roc']

In [33]:
grouped_model_feat = pd.concat([group_train,group_train_std,group_val,group_val_std,
                group_test,group_test_std,group_ext,group_ext_std]+tmp_list,axis=1)
grouped_model_feat.columns = ['train_roc','train_roc_std','val_roc','val_roc_std',
              'test_roc','test_roc_std', 'ext_roc','ext_roc_std'] + \
            ['rmSim_0.6','rmSim_0.6_std','rmSim_0.7','rmSim_0.7_std',
             'rmSim_0.8','rmSim_0.8_std', 'rmSim_0.9','rmSim_0.9_std']

In [34]:
grouped_model_feat.to_csv('grouped_model_feat.csv')

In [8]:
grouped_model_feat = pd.read_csv('grouped_model_feat.csv')

In [9]:
grouped_model_feat

,model,feat,train_roc,train_roc_std,val_roc,val_roc_std,test_roc,test_roc_std,ext_roc,ext_roc_std,rmSim_0.6,rmSim_0.6_std,rmSim_0.7,rmSim_0.7_std,rmSim_0.8,rmSim_0.8_std,rmSim_0.9,rmSim_0.9_std
0,DMPNN,NaN,0.978213,0.006126,0.978213,0.006126,0.930903,0.028380,0.880134,0.017685,0.940741,0.067281,0.844842,0.041713,0.826310,0.028214,0.870979,0.021187
1,DMPNN,AtomPairFP,0.998923,0.000602,0.998923,0.000602,0.935529,0.022783,0.901934,0.020097,0.185185,0.055212,0.790737,0.058572,0.826009,0.031065,0.890332,0.021440
2,DMPNN,Autocorr,0.994881,0.005635,0.994881,0.005635,0.930406,0.026056,0.824148,0.032109,0.540741,0.087646,0.508842,0.080051,0.693052,0.045660,0.801075,0.033656
3,DMPNN,AvalonFP,0.994742,0.006112,0.994742,0.006112,0.929702,0.021141,0.608466,0.029307,0.196296,0.118158,0.214947,0.046920,0.445371,0.031920,0.585146,0.027208
4,DMPNN,Charge,0.985097,0.007047,0.985097,0.007047,0.938356,0.021989,0.870815,0.004982,0.355556,0.094346,0.674105,0.064484,0.748732,0.016108,0.852359,0.006847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,xgb,Property,0.999199,0.000135,0.999199,0.000135,0.885889,0.028687,0.800472,0.015439,0.348148,0.061975,0.459158,0.059687,0.627061,0.027708,0.778889,0.015453
147,xgb,PubChemFP,0.995366,0.000526,0.995366,0.000526,0.934550,0.023271,0.767041,0.047619,0.081481,0.066254,0.320842,0.160666,0.555164,0.080624,0.735595,0.052189
148,xgb,RDkitFP,0.999804,0.000042,0.999804,0.000042,0.942591,0.023015,0.806203,0.019396,0.111111,0.000000,0.425263,0.071668,0.655408,0.031133,0.784425,0.022249
149,xgb,Topology,0.996849,0.000354,0.996849,0.000354,0.878019,0.029689,0.802475,0.016137,0.200000,0.132508,0.609158,0.049325,0.652798,0.027306,0.776328,0.016966


In [17]:
eva_csv_name = 'eva_with_rmSim.csv'
df = pd.read_csv(eva_csv_name)
col_names = df['col_names'].values
m_list = [];data_list = [];feat_list = [];fold_list = []
for col in col_names:
    m,tmp = col.split('/')
    tmp = tmp.replace('.csv','').replace('p_Ext_rm_sim_0_','p_ExtRmSim0')
    _,dt,ft,fold = tmp.split('_')
    m_list.append(m);data_list.append(dt);feat_list.append(ft);fold_list.append(fold)
df['model'] = m_list
df['data'] = data_list
df['feat'] = feat_list
df['fold'] = fold_list

In [12]:
#df.columns.to_list()
#['col_names', 'roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE',
# 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']

In [48]:
def group_model_feat_for_one_performance_sort(xeva = 'roc'):
    group_test = test_df.groupby(['model','feat']).mean()[xeva]
    group_test_std = test_df.groupby(['model','feat']).std()[xeva]
    group_ext = ext_df.groupby(['model','feat']).mean()[xeva]
    group_ext_std = ext_df.groupby(['model','feat']).std()[xeva]

    group_train = train_df.groupby(['model','feat']).mean()[xeva]
    group_train_std = train_df.groupby(['model','feat']).std()[xeva]
    group_val = val_df.groupby(['model','feat']).mean()[xeva]
    group_val_std = val_df.groupby(['model','feat']).std()[xeva]

    dfl = [pd.DataFrame() for i in [6,7,8,9]] 
    groupl = [group_val for i in [6,7,8,9]] 
    group_stdl = [group_val_std for i in [6,7,8,9]] 
    tmp_list = []
    for i,sim in enumerate([6,7,8,9]):
        #dfl[0]=df[df['data']=='train']
        dfl[i]=df[df['data']==f'ExtRmSim0{sim}']
        groupl[i] = dfl[i].groupby(['model','feat']).mean()[xeva]
        group_stdl[i] = dfl[i].groupby(['model','feat']).std()[xeva]
        tmp_list.append(groupl[i]);tmp_list.append(group_stdl[i])
    grouped_model_feat = pd.concat([group_train,group_train_std,group_val,group_val_std,
                group_test,group_test_std,group_ext,group_ext_std]+tmp_list,axis=1)
    grouped_model_feat.columns = [f'train_{xeva}',f'train_{xeva}_std',f'val_{xeva}',f'val_{xeva}_std',
              f'test_{xeva}',f'test_{xeva}_std', f'ext_{xeva}',f'ext_{xeva}_std'] + \
            ['rmSim_0.6','rmSim_0.6_std','rmSim_0.7','rmSim_0.7_std',
             'rmSim_0.8','rmSim_0.8_std', 'rmSim_0.9','rmSim_0.9_std']
    grouped_model_feat.to_csv(f'grouped_{xeva}.csv')
    grouped_model_feat = pd.read_csv(f'grouped_{xeva}.csv')
    grouped_sort=grouped_model_feat.sort_values(by=[f'ext_{xeva}',f'test_{xeva}',],ascending=False)
    grouped_sort.to_csv(f'grouped_sort_{xeva}.csv',index=False)
    return grouped_sort

In [18]:
test_df=df[df['data']=='test']
ext_df=df[df['data']=='Ext']
train_df=df[df['data']=='train']
val_df=df[df['data']=='train']

In [49]:
grouped_sort_roc = group_model_feat_for_one_performance_sort(xeva = 'roc')

In [50]:
grouped_sort_acc = group_model_feat_for_one_performance_sort(xeva = 'accuracy')

In [51]:
grouped_sort_MCC = group_model_feat_for_one_performance_sort(xeva = 'MCC')
grouped_sort_BA = group_model_feat_for_one_performance_sort(xeva = 'BA')

In [46]:
def group_model_for_one_performance_sort(xeva = 'roc'):
    group_test = test_df.groupby(['model']).mean()[xeva]
    group_test_std = test_df.groupby(['model']).std()[xeva]
    group_ext = ext_df.groupby(['model']).mean()[xeva]
    group_ext_std = ext_df.groupby(['model']).std()[xeva]

    group_train = train_df.groupby(['model']).mean()[xeva]
    group_train_std = train_df.groupby(['model']).std()[xeva]
    group_val = val_df.groupby(['model']).mean()[xeva]
    group_val_std = val_df.groupby(['model']).std()[xeva]

    dfl = [pd.DataFrame() for i in [6,7,8,9]] 
    groupl = [group_val for i in [6,7,8,9]] 
    group_stdl = [group_val_std for i in [6,7,8,9]] 
    tmp_list = []
    for i,sim in enumerate([6,7,8,9]):
        #dfl[0]=df[df['data']=='train']
        dfl[i]=df[df['data']==f'ExtRmSim0{sim}']
        groupl[i] = dfl[i].groupby(['model']).mean()[xeva]
        group_stdl[i] = dfl[i].groupby(['model']).std()[xeva]
        tmp_list.append(groupl[i]);tmp_list.append(group_stdl[i])
    grouped_model_feat = pd.concat([group_train,group_train_std,group_val,group_val_std,
                group_test,group_test_std,group_ext,group_ext_std]+tmp_list,axis=1)
    grouped_model_feat.columns = [f'train_{xeva}',f'train_{xeva}_std',f'val_{xeva}',f'val_{xeva}_std',
              f'test_{xeva}',f'test_{xeva}_std', f'ext_{xeva}',f'ext_{xeva}_std'] + \
            ['rmSim_0.6','rmSim_0.6_std','rmSim_0.7','rmSim_0.7_std',
             'rmSim_0.8','rmSim_0.8_std', 'rmSim_0.9','rmSim_0.9_std']
    grouped_model_feat.to_csv(f'grouped_{xeva}.csv')
    grouped_model_feat = pd.read_csv(f'grouped_{xeva}.csv')
    grouped_sort=grouped_model_feat.sort_values(by=[f'ext_{xeva}',f'test_{xeva}',],ascending=False)
    grouped_sort.to_csv(f'grouped_model_sort_{xeva}.csv',index=False)
    return grouped_sort

In [47]:
grouped_model_roc = group_model_for_one_performance_sort(xeva = 'roc')